## Exercícios básicos sobre LCEL - LangChain
___

## =============================================================================
## CONCEITOS FUNDAMENTAIS DO LANGCHAIN
## =============================================================================

"""
🎯 CORE COMPONENTS DE UMA APLICAÇÃO LANGCHAIN:

1. ENTRADA (Input) - Dicionário simples com os dados de entrada
2. PROMPT - Template formatado usando ChatPromptTemplate.from_messages
3. MODELO - LLM (Large Language Model) - OpenAI no nosso caso
4. PARSER DE SAÍDA - StrOutputParser() ou with_structured_output()

🔗 LCEL (LangChain Expression Language):
- Sintaxe: input | prompt | llm | output_parser
- O operador "|" (pipe) conecta os componentes em sequência
- Cada componente processa a saída do anterior
"""

In [ ]:
#* --- CÉLULA DE SETUP IDEAL ---

import os
from dotenv import find_dotenv, load_dotenv
from langchain_openai import ChatOpenAI

# Desabilita o tracing para manter o output limpo nos exercícios
# Dica: Para depurar chains complexas, mude para "true" e configure o LangSmith!
os.environ["LANGCHAIN_TRACING_V2"] = "false"

# Carrega as variáveis de ambiente do arquivo .env
# A função retorna True se encontrou o arquivo, False caso contrário.
if not load_dotenv(find_dotenv()):
    print("Arquivo .env não encontrado. Verificando variáveis de ambiente do sistema.")

# Valida a chave da API e instancia o LLM de forma segura e limpa
# O LangChain busca a chave do ambiente automaticamente. Não é necessário
# carregar a chave em uma variável ou passá-la explicitamente.
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("A variável de ambiente OPENAI_API_KEY não foi encontrada.")

# Instanciação simplificada:
llm = ChatOpenAI(model="gpt-4o-mini")

print("✅ Configuração do LLM realizada com sucesso!")
# Para um teste rápido, você pode descomentar a linha abaixo:
# print(llm.invoke("Diga olá em português.").content)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import Runnable, RunnableSerializable
from pydantic import BaseModel, Field
from typing import List, Dict, Any

In [ ]:
# ===============================================================================
# EXERCÍCIO 1: CHAIN BÁSICA - {'produto': 'café} e retorna uma frase de marketing
# ===============================================================================
# Defina o LLM com um pouco de temperatura para mais criatividade
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

def marketing_chain(llm: ChatOpenAI) -> str:
    """
    Cria uma cadeia simples que gera uma frase de marketing para um produto.

    Args:
        llm (Runnable): O modelo de linguagem a ser utilizado.

    Returns:
        str: A frase de marketing gerada pelo modelo.
    """
    # 1. Entrada - Dicionário simples
    input_data = {"produto": "café"}

    # 2. Defina o prompt usando ChatPromptTemplate
    prompt = ChatPromptTemplate([
        ("system", "Você é um especialista em marketing."),
        ("user", "Crie uma frase de marketing para o seguinte produto: {produto}")
    ])

    # 3. Defina o parser de saída
    parser = StrOutputParser()

    # 4. Criando a chain com LCEL - Tipagem correta
    chain: RunnableSerializable[Dict[str, Any], str] = prompt | llm | parser # type: ignore

    # 5. Execute a chain com os dados de entrada
    result = chain.invoke(input_data)
    print(f"📝 Entrada: {input_data['produto']}")
    print(f"🔄 Processamento: Prompt → LLM → Parser")
    print(f"✨ Resultado: {result}")

    return result

### Exercício 1:

* entrada: `{"produto":"café"}`
* output: frase de marketing sobre café

In [ ]:
marketing_café = marketing_chain(llm)

### Exercício 2:

* entrada: `{"palavra":"gato", "idioma":"inglês"}`
* output: tradução da palavra de entrada para o idioma

In [ ]:
# ===============================================================================
# EXERCÍCIO 2: CHAIN BÁSICA - tradução de palavras {'palavra': 'gato', 'idioma': 'inglês'}
# ===============================================================================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)

def chain_tradutor(llm: ChatOpenAI) -> str:
    """
    Traduz uma palavra para um idioma especificado usando uma cadeia simples.

    Args:
        palavra (str): A palavra a ser traduzida.
        idioma (str): O idioma para o qual a palavra deve ser traduzida.

    Returns:
        str: Palavra traduzida na língua especificada.
    """
    # 1. Entrada - Dicionário simples
    input_data = {
        "palavra": "gato",
        "idioma": "inglês"
    }

    # 2. Defina o prompt usando ChatPromptTemplate
    prompt = ChatPromptTemplate([
        ("system", "Você é um tradutor experiente."),
        ("user", "Traduza a seguinte palavra: '{palavra}' para {idioma}.")
    ])
    # 3. Defina o parser de saída
    parser = StrOutputParser()

    # 4. Criando a chain co LCEL
    chain = prompt | llm | parser # type: ignore

    # 5. Execute a chain com os dados de entrada
    result = chain.invoke(input_data) # type: ignore

    # Processando saída
    print(f"📝 Entrada: {input_data}")
    print(f"🔄 Processamento: Prompt → LLM → Parser")
    print(f"✨ Resultado: {result}")

    return result

In [ ]:
traducao_palavra = chain_tradutor(llm)

### Escreva uma chain que recebe `{"palavra":"saudade"}` e retorna a sua definição

In [ ]:
# Defina uma chain que recebe {"palavra":"saudade"} e retorna a sua definição
# Definindo uma temperatura intermediária para um equilíbrio entre criatividade e precisão
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)

# 1. Entrada - Dicionário simples
input_data = {"palavra":"saudade"}

# 2. Defina o prompt usando ChatPromptTemplate
prompt = ChatPromptTemplate([
        ("system", "Você é um especialista em significado de palavras."),
        ("user", "Escreva a definição da palavra : {palavra}.")
    ])
    # 3. Defina o parser de saída
parser = StrOutputParser()

# 4. Criando a chain co LCEL
chain = prompt | llm | parser # type: ignore

# 5. Execute a chain com os dados de entrada
result = chain.invoke(input_data) # type: ignore

# Processando saída
print(f"📝 Entrada: {input_data}")
print(f"🔄 Processamento: Prompt → LLM → Parser")
print(f"✨ Resultado: {result}")

### Uma chain que recebe:

`{"filme": "O Poderoso Chefão"}` e retorna o nome do diretor (use um Pydantic Model para a saída).

In [ ]:
class DirectorMovie(BaseModel):
    nome: str = Field(..., description="Nome do diretor do filme")

# Modelo de linguagem com temperatura equilibrada para precisão e criatividade.
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.5)
# 1. Entrada - Dicionário simples
input_data = {"filme": "O Poderoso Chefão"}

# 2. Defina o prompt usando ChatPromptTemplate
prompt = ChatPromptTemplate([
        ("system", "Você é um especialista em filmes."),
        ("user", "Quem é o diretor do filme : {filme}? Responda apenas com o nome.")
    ])
# 3. Defina o parser de saída usando Pydantic
llm_structured = llm.with_structured_output(DirectorMovie) # type: ignore

# 4. Criando a chain co LCEL
chain = prompt | llm_structured # type: ignore

# 5. Execute a chain com os dados de entrada
result = chain.invoke(input_data) # type: ignore

# Processando saída
print(f"📝 Entrada: {input_data}")
print(f"🔄 Processamento: Prompt → LLM → Parser")
print(f"✨ Resultado: {result.nome}") #type: ignore


In [ ]:
def exercicio_2():
    """
    📚 OBJETIVO: Trabalhar com prompts mais complexos e múltiplas variáveis
    
    CONCEITO: Como o LangChain passa dados entre componentes
    - O prompt recebe um dicionário e formata as variáveis
    - O LLM recebe o prompt formatado
    - O parser limpa a resposta do LLM
    """
    
    # 1. ENTRADA - Múltiplas variáveis
    entrada = {
        "produto": "smartphone",
        "marca": "Apple",
        "preco": "R$ 3.500",
        "publico": "jovens profissionais"
    }
    
    # 2. PROMPT - Template mais complexo
    prompt = ChatPromptTemplate([
        ("system", "Você é um especialista em marketing digital."),
        ("human", """
        Crie uma descrição de produto para vendas online:
        
        Produto: {produto}
        Marca: {marca}
        Preço: {preco}
        Público-alvo: {publico}
        
        A descrição deve ser persuasiva e focada no público-alvo.
        """)
    ])
    
    # 3. PARSER DE SAÍDA
    parser = StrOutputParser()
    
    # 4. CHAIN
    chain = prompt | llm | parser # type: ignore
    
    # 5. EXECUÇÃO
    resultado = chain.invoke(entrada) # type: ignore
    
    print(f"📝 Entrada: {entrada}")
    print(f"✨ Descrição do Produto:\n{resultado}")
    
    return resultado

# Executar exercício 2
resultado_2 = exercicio_2()

In [ ]:
# =============================================================================
# EXERCÍCIO 6: CHAIN COM SAÍDA ESTRUTURADA (PYDANTIC)
# =============================================================================

print("\n" + "="*60)
print("🚀 EXERCÍCIO 6: CHAIN COM SAÍDA ESTRUTURADA")
print("="*60)

def exercicio_6():
    """
    📚 OBJETIVO: Usar with_structured_output() para dados estruturados

    VANTAGEM: Em vez de string livre, obtemos um objeto Python estruturado
    - Validação automática dos dados
    - Acesso direto aos campos
    - Melhor para integrações com outras partes do código
    """

    # 1. MODELO PYDANTIC - Define a estrutura da saída
    class AnaliseTexto(BaseModel):
        """Modelo para análise estruturada de texto"""
        sentimento: str = Field(description="Sentimento: positivo, negativo ou neutro")
        confianca: float = Field(description="Nível de confiança de 0 a 1")
        palavras_chave: List[str] = Field(description="Lista de palavras-chave importantes")
        resumo: str = Field(description="Resumo em uma frase")

    # 2. ENTRADA
    entrada = {
        "texto": """
        Estou absolutamente encantado com este produto! A qualidade superou todas as minhas 
        expectativas. O atendimento ao cliente foi excepcional, e a entrega foi mais rápida 
        do que prometido. Recomendo fortemente para qualquer pessoa que esteja considerando 
        esta compra. Definitivamente comprarei novamente!
        """
    }

    # 3. PROMPT
    prompt = ChatPromptTemplate([
        ("system", "Você é um especialista em análise de sentimentos e texto."),
        ("human", "Analise o seguinte texto: {texto}")
    ])

    # 4. LLM COM SAÍDA ESTRUTURADA - Substitui o StrOutputParser
    llm_estruturado = llm.with_structured_output(AnaliseTexto) # type: ignore

    # 5. CHAIN
    chain = prompt | llm_estruturado # type: ignore

    # 6. EXECUÇÃO
    resultado = chain.invoke(entrada) # type: ignore

    print(f"📝 Texto analisado: {entrada['texto'][:100]}...")
    print(f"🎯 Sentimento: {resultado.sentimento}")
    print(f"📊 Confiança: {resultado.confianca}")
    print(f"🏷️  Palavras-chave: {resultado.palavras_chave}")
    print(f"📋 Resumo: {resultado.resumo}")

    return resultado # type: ignore

# Executar exercício 3
resultado_6 = exercicio_6() # type: ignore